# Predictor de Popularidade de Músicas

**Índice**<a id='toc0_'></a>    
1. [Introdução](#toc1_)    
2. [Preparação do notebook para R e Python em sumultâneo](#toc2_)    
3. [Script R Madalena](#toc3_)    
4. [Importação dos módulos](#toc4_)    
5. [Exploração dos dados em bruto](#toc5_)    
5.1. [Upload dos dados](#toc5_1_)    
5.2. [Criação do dataframe](#toc5_2_)    
5.3. [Visualização do dataframe em bruto e de algumas métricas](#toc5_3_)    
6. [Análise Exploratória](#toc6_)    
6.1. [Distribuição dos valores em cada coluna](#toc6_1_)    
7. [Pré-processamento dos dados](#toc7_)    
7.1. [Identificação de Valores Ausentes](#toc7_1_)    
7.2. [Remoção de musicas com tempo 0 e sem palavras](#toc7_2_)    
7.3. [Identificação e remoção de duplicados](#toc7_3_)    
7.4. [Definir variáveis categóricas e variáveis numéricas](#toc7_4_)    
7.5. [Definir o índice](#toc7_5_)    
7.6. [Método do intervalo interquartil (IQR) para filtrar outliers](#toc7_6_)    
7.7. [Estatísticas do dataset](#toc7_7_)    
7.8. [Standerização dos dados](#toc7_8_)    
8. [Divisão em conjunto de treino e conjunto de teste](#toc8_)    
9. [Implementação de algoritmos de Aprendizagem Computacional](#toc9_)    
9.1. [Random Forest](#toc9_1_)    
9.1.1. [Treino inicial do modelo](#toc9_1_1_)    
9.1.2. [Afinação dos hiperparâmetros com o GridSearchCV](#toc9_1_2_)    
9.1.3. [Configurar e executar o GridSearchCV](#toc9_1_3_)    
9.1.4. [Treino do modelo com os melhores hiperparâmetros](#toc9_1_4_)    
10. [Resultados](#toc10_)    
11. [Considerações finais](#toc11_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Introdução](#toc0_)

## 2. <a id='toc2_'></a>[Preparação do notebook para R e Python em sumultâneo](#toc0_)

In [ ]:
# ! R -e "install.packages('IRkernel', repos = 'http://cran.us.r-project.org');IRkernel::installspec()"

In [ ]:
# ! pip install rpy2

In [ ]:
import rpy2
print(rpy2.__version__)

import rpy2.situation
for row in rpy2.situation.iter_info():
    print(row)

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import StrVector

# Importing necessary R packages
r_base = importr('base')
r_utils = importr('utils')
pls = importr('pls')
psych = importr('psych')
readr = importr('readr')
tree = importr('tree')
caret = importr('caret')
gbm = importr('gbm')
rpart = importr('rpart')
rpart_plot = importr('rpart.plot')
randomForest = importr('randomForest')
pROC = importr('pROC')
ggplot2 = importr('ggplot2')
reshape2 = importr('reshape2')

In [ ]:
%load_ext rpy2.ipython

In [ ]:
#%%R -i df_no_outliers
#head(df_no_outliers) #Exemplo de utilização

## 3. <a id='toc3_'></a>[Script R Madalena](#toc0_)

In [ ]:
%%R
data=read.csv("data/song_data.csv")
head(data)
data.numeric <- data[,sapply(data, is.numeric)]

head(data.numeric)
install.packages("pls")
library(pls)
library(psych) 
library(readr)
library(tree)
library(caret)
library(gbm)
library(rpart)
library(rpart.plot)
library(randomForest)
library(caret)
library(pROC)
library(ggplot2)
library(reshape2)

In [ ]:
%%R 
quartiles <- quantile(data$song_popularity, probs = c(0, 0.25, 0.5, 0.75, 0.95), na.rm = TRUE)
breaks <- c(quartiles[1:5], Inf)
labels <- c("Não_popular", "Pouco_popular", "Rádio_amigável", "Popular", "Viral")
data$song_category <- cut(data$song_popularity, breaks = breaks, labels = labels, include.lowest = TRUE)
summary(data$song_category)
data$song_name<-NULL
head(data)

In [ ]:
%%R
hist(data$song_duration_ms)
hist(data$acousticness)
hist(data$tempo)
correlation_matrix <- cor(data.numeric)
correlation_data <- melt(correlation_matrix)

In [ ]:
%%R
ggplot(correlation_data, aes(Var1, Var2, fill = value)) +
  geom_tile(color = "white") +
  scale_fill_gradient2(low = "blue", high = "red", mid = "white", 
                       midpoint = 0, limit = c(-1,1), 
                       space = "Lab", name="Correlation") +
  geom_text(aes(label = round(value, 2)), color = "black", size = 3) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, vjust = 1, size = 10, hjust = 1)) +
  coord_fixed() +
  labs(title = "Correlation Heatmap", x = "Variables", y = "Variables")

In [ ]:
%%R
boxplot(data$song_duration_ms)

boxplot(data$instrumentalness)
ggplot(data, aes(x = song_category, y = song_duration_ms)) +
  geom_boxplot() +
  labs(title = "Box plot of Your Variable by Song Category",
       x = "Song Category",
       y = "song_duration_ms")


threshold <- 1000000  # limite dos outliers (limite)
data <- data[data$song_duration_ms <= threshold, ]
dim(data)
18835-18831 ##4 outliers eliminados

#####Variavel acousticness

ggplot(data, aes(x = song_category, y = acousticness)) +
  geom_boxplot() +
  labs(title = "Box plot of Your Variable by Song Category",
       x = "Song Category",
       y = "Acousticness")

# Definir o limite (threshold)
threshold1 <- 0.98
data <- data[!(data$song_category == "Popular" & data$acousticness > threshold1), ]
head(data)
18835-18814 ##21 outliers eliminados


#####Variavel Energy
##Não é preciso eliminar outliers

ggplot(data, aes(x = song_category, y = energy
)) +
  geom_boxplot() +
  labs(title = "Box plot of Your Variable by Song Category",
       x = "Song Category",
       y = "Energy")


#####Variavel Speechiness

ggplot(data, aes(x = song_category, y = speechiness
)) +
  geom_boxplot() +
  labs(title = "Box plot of Your Variable by Song Category",
       x = "Song Category",
       y = "Speechiness")

threshold1 <- 0.9  
threshold2 <- 0.9 
threshold3 <- 0.75
threshold4 <- 0.5
data <- data[!(data$song_category == "Not_popular" & data$speechiness > threshold1), ]
data <- data[!(data$song_category == "Low_popularity" & data$speechiness > threshold2), ]
data <- data[!(data$song_category == "Radio_friendly" & data$speechiness > threshold3), ]
data <- data[!(data$song_category == "Popular" & data$speechiness > threshold4), ]

dim(data)
18831-18794 ##37 outliers eliminados

####Variavel Loudness

ggplot(data, aes(x = song_category, y = loudness
)) +
  geom_boxplot() +
  labs(title = "Box plot of Your Variable by Song Category",
       x = "Song Category",
       y = "Loudness")
threshold1=-35
data <- data[!(data$loudness < threshold1), ]
dim(data)
18831-18788 ##43 outliers eliminados

####Variavel Speechiness

ggplot(data, aes(x = song_category, y = speechiness
)) +
  geom_boxplot() +
  labs(title = "Box plot of Your Variable by Song Category",
       x = "Song Category",
       y = "Speechiness")

threshold1=0.85
data <- data[!(data$speechiness > threshold1), ]
dim(data)
18831-18783 ##48 outliers eliminados


##Variavel Tempo
ggplot(data, aes(x = song_category, y = tempo
)) +
  geom_boxplot() +
  labs(title = "Box plot of Your Variable by Song Category",
       x = "Song Category",
       y = "Tempo")
threshold1=230
threshold2=0
data <- data[!(data$tempo > threshold1), ]
data <- data[!(data$tempo <= threshold2), ]
dim(data)
18831-18780 ##51 outliers eliminados


##Variavel Danceability
ggplot(data, aes(x = song_category, y = danceability
)) +
  geom_boxplot() +
  labs(title = "Box plot of Your Variable by Song Category",
       x = "Song Category",
       y = "Danceability")


In [ ]:
%%R
df <- data.frame(data)

## 4. <a id='toc4_'></a>[Importação dos módulos](#toc0_)

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt


## 5. <a id='toc5_'></a>[Exploração dos dados em bruto](#toc0_)

### 5.1. <a id='toc5_1_'></a>[Upload dos dados](#toc0_)

In [ ]:
data = pd.read_csv('data/song_data.csv')

### 5.2. <a id='toc5_2_'></a>[Criação do dataframe](#toc0_)

In [ ]:
df_raw = pd.DataFrame(data)

### 5.3. <a id='toc5_3_'></a>[Visualização do dataframe em bruto e de algumas métricas](#toc0_)

In [ ]:
df_no_outliers = %R df

In [ ]:
df_raw.info()

In [ ]:
df_raw.shape

## 6. <a id='toc6_'></a>[Análise Exploratória](#toc0_)

### 6.1. <a id='toc6_1_'></a>[Distribuição dos valores em cada coluna](#toc0_)

In [ ]:
col_names = df_raw.columns

col_names

In [ ]:
col_counts = {}

for col in col_names:
    col_counts[col] = df_raw[col].value_counts()

col_counts


## 7. <a id='toc7_'></a>[Pré-processamento dos dados](#toc0_)

### 7.1. <a id='toc7_1_'></a>[Identificação de Valores Ausentes](#toc0_)

In [ ]:
df_raw.isnull().sum()

### 7.2. <a id='toc7_2_'></a>[Remoção de musicas com tempo 0 e sem palavras](#toc0_)

In [ ]:
df_raw.drop(df_raw[(df_raw['tempo']==0) & (df_raw['speechiness']==0)].index, inplace=True)

### 7.3. <a id='toc7_3_'></a>[Identificação e remoção de duplicados](#toc0_)

In [ ]:
df_raw.drop_duplicates(inplace=True)

In [ ]:
df_raw.shape

### 7.4. <a id='toc7_4_'></a>[Definir variáveis categóricas e variáveis numéricas](#toc0_)

In [ ]:
from sklearn.compose import make_column_selector as selector
numerical_columns_selector = selector(dtype_exclude='category')
categorical_columns_selector = selector(dtype_include='category')

numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)

### 7.5. <a id='toc7_5_'></a>[Definir o índice](#toc0_)

In [ ]:
df_raw.set_index('song_name', inplace=True)

### 7.6. <a id='toc7_6_'></a>[Método do intervalo interquartil (IQR) para filtrar outliers](#toc0_)

In [ ]:
numeric_columns = df_raw.select_dtypes(include='number')

# Calcula Q1, Q3 e IQR para cada coluna numérica
Q1 = numeric_columns.quantile(0.25)
Q3 = numeric_columns.quantile(0.75)
IQR = Q3 - Q1

# Define os limites para identificar outliers
lower_bound = Q1 - 1.5 * IQR 
upper_bound = Q3 + 1.5 * IQR

# Identifica e remove outliers
outliers = ((numeric_columns < lower_bound) | (numeric_columns > upper_bound)).any(axis=1)
df_no_outliers = df_raw[~outliers]

In [ ]:
df_no_outliers

In [ ]:
df_no_outliers.shape

Inferência:

Antes da remoção de duplicados e outliers, o conjunto de dados tinha 18835 amostras.

Após a remoção de duplicados e outliers, o conjunto de dados ficou com 8609 amostras.

### 7.7. <a id='toc7_7_'></a>[Estatísticas do dataset](#toc0_)

In [ ]:
import matplotlib.pyplot as plt

# Definindo as cores
colors = ['green', 'red']  # Exemplo de cores: azul e laranja

plt.title('Dataset Final')
plt.pie([df_no_outliers.shape[0], df_raw.shape[0]-df_no_outliers.shape[0]], 
        radius=1, 
        labels=['Mantido', 'Excluído'], 
        counterclock=False, 
        autopct='%1.1f%%', 
        pctdistance=0.9, 
        explode=[0.02, 0], 
        shadow=False,
        colors=colors)  # Adicionando a lista de cores
plt.show()


In [ ]:
df_no_outliers.describe()

In [ ]:
plt.figure(figsize=[8,4])
sns.distplot(df_no_outliers['song_popularity'], color='b',hist_kws=dict(edgecolor="black", linewidth=2), bins=50)
plt.title('Target Variable Distribution - Song Popularity')
plt.show()

In [ ]:
plt.figure(figsize=[8,4])
sns.distplot(df_no_outliers['song_popularity'], color='black',hist_kws=dict(edgecolor="black", linewidth=2), bins=50)
plt.title('Target Variable Distribution - Song Popularity')
plt.show()

In [ ]:
# Configurar a figura com duas subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))

# Plotar o primeiro gráfico
sns.histplot(df_no_outliers['song_popularity'], color='black', edgecolor="black", linewidth=2, bins=50, ax=axes[0])
axes[0].set_title('Target Variable Distribution - Song Popularity')

# Plotar o segundo gráfico
sns.histplot(df_no_outliers['song_popularity'], color='b', edgecolor="black", linewidth=2, bins=50, ax=axes[1])
axes[1].set_title('Target Variable Distribution - Song Popularity (No Outliers)')

# Mostrar o gráfico
plt.tight_layout()
plt.show()

In [ ]:
# Splitting the data intro training & testing sets

from sklearn.model_selection import train_test_split

m=[]
for i in df_no_outliers.columns.values:
    m.append(i.replace(' ','_'))
    
df_no_outliers.columns = m
X = df_no_outliers[['song_duration_ms', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'audio_mode', 'speechiness', 'tempo', 'time_signature', 'audio_valence']]
X.loc[:, ['key', 'audio_mode', 'time_signature']] = X.loc[:, ['key', 'audio_mode', 'time_signature']].astype('category')
Y = df_no_outliers["song_popularity"]



### 7.8. <a id='toc7_8_'></a>[Standerização dos dados](#toc0_)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

In [ ]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer([
    ('One-Hot-Encoder', categorical_preprocessor, categorical_columns),
    ('Standard-Scaler', numerical_preprocessor, numerical_columns)])

## 8. <a id='toc8_'></a>[Divisão em conjunto de treino e conjunto de teste](#toc0_)

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=42)
Train_X.reset_index(drop=True,inplace=True)

print('Conjunto original (100%): ',X.shape,Y.shape,'\nConjunto de treino (80%): ',Train_X.shape,Train_Y.shape,'\nConjunto de teste (20%): ', Test_X.shape,'', Test_Y.shape)

## 9. <a id='toc9_'></a>[Implementação de algoritmos de Aprendizagem Computacional](#toc0_)

### 9.1. <a id='toc9_1_'></a>[Random Forest](#toc0_)

#### 9.1.1. <a id='toc9_1_1_'></a>[Treino inicial do modelo](#toc0_)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn import set_config

# Escolha RandomForestClassifier para problemas de classificação e RandomForestRegressor para regressão
model = make_pipeline(preprocessor,RandomForestRegressor(random_state=21))  # ou RandomForestRegressor
set_config(display='diagram')

# Treinar o modelo
model.fit(Train_X, Train_Y)

# Fazer previsões
Pred_Y = model.predict(Test_X)



In [ ]:
model

In [ ]:
def modelresults(predictions):
    mae = mean_absolute_error(Test_Y, predictions)
    mse = mean_squared_error(Test_Y, predictions)
    r2 = r2_score(Test_Y, predictions)
    
    print('Mean absolute error on model is {:.4f}'.format(mae))
    print('')
    print('Mean squared error on model is {:.4f}'.format(mse))
    print('')
    print('The r2 score on model is {:.4f}'.format(r2))

In [ ]:
modelresults(Pred_Y)

#### 9.1.2. <a id='toc9_1_2_'></a>[Afinação dos hiperparâmetros com o GridSearchCV](#toc0_)

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],      # Número de árvores na floresta
    'max_features': ['auto', 'sqrt', 'log2'],  # Número de features a serem consideradas para divisão
    'max_depth': [None, 10, 20, 30],      # Profundidade máxima da árvore
    'min_samples_split': [2, 5, 10],      # Número mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4]         # Número mínimo de amostras em um nó folha
}


#### 9.1.3. <a id='toc9_1_3_'></a>[Configurar e executar o GridSearchCV](#toc0_)

In [ ]:
# Configurar o GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=21), param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Executar o GridSearchCV
grid_search.fit(Train_X, Train_Y)

# Obter os melhores hiperparâmetros
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_
print(f"Melhor estimador: {best_estimator}")
print(f'Melhores hiperparâmetros: {best_params}')


#### 9.1.4. <a id='toc9_1_4_'></a>[Treino do modelo com os melhores hiperparâmetros](#toc0_)

In [ ]:
# Criar um novo modelo com os melhores hiperparâmetros
#best_model = RandomForestClassifier(**best_params, random_state=42)  # ou RandomForestRegressor
best_model = RandomForestRegressor(**best_params, random_state=42)  # ou RandomForestRegressor

# Treinar o modelo
best_model.fit(Train_X, Train_Y)

# Fazer previsões
best_y_pred = best_model.predict(Test_X)


In [ ]:
modelresults(best_y_pred)

## 10. <a id='toc10_'></a>[Resultados](#toc0_)

## 11. <a id='toc11_'></a>[Considerações finais](#toc0_)